In [9]:
import torch
from metric import cosine_similarity_matrix
import numpy as np
dataset_name = 'en'
candidate_jd_embeddings = torch.load(f'outputs/{dataset_name}_candidate_jd_embeddings.pt')
target_jd_embeddings = torch.load(f'outputs/{dataset_name}_target_jd_embeddings.pt')
sim_bert = cosine_similarity_matrix(target_jd_embeddings.detach().cpu().numpy(), candidate_jd_embeddings.detach().cpu().numpy())
np.save(f'outputs/{dataset_name}_sim_bert.npy', sim_bert)

candidate_jd_embeddings = torch.load(f'outputs/{dataset_name}_candidate_jd_raw_embeddings.pt')
target_jd_embeddings = torch.load(f'outputs/{dataset_name}_target_jd_raw_embeddings.pt')
sim_raw_bert = cosine_similarity_matrix(target_jd_embeddings.detach().cpu().numpy(), candidate_jd_embeddings.detach().cpu().numpy())
np.save(f'outputs/{dataset_name}_sim_raw_bert.npy', sim_raw_bert)
sim_edit = np.load('outputs/en_sim-Levenshtein.npy')

/tmp/ipykernel_1600604/3581502509.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  candidate_jd_embeddings = torch.load(f'outputs/{dataset_name}_candidate_jd_embeddings.p

In [12]:
sim_word_edit = np.load('outputs/en_sim-word_Levenshtein.npy')
sim_word_edit.shape

(133, 7619)

In [10]:
import pandas as pd
candidate_jd = pd.read_csv('data/en/candidate_jd.csv')
target_jd = pd.read_csv('data/en/target_jd.csv')

bert_ans = np.argsort(sim_bert)
bert_raw_ans = np.argsort(sim_raw_bert)

edit_ans = np.argsort(-sim_edit)
edit_word_ans = np.argsort(-sim_word_edit)


sort_sim_bert = np.sort(sim_bert, axis=-1)
sort_sim_bert_raw = -np.sort(-sim_raw_bert, axis=-1)

sort_sim_edit = np.sort(sim_edit, axis=-1)


In [11]:
target_jd['sim_bert'] = [i for i in bert_ans]
target_jd['sim_bert_value'] = [i for i in sort_sim_bert]
target_jd['sim_bert_raw_value'] = [i for i in sort_sim_bert_raw]

target_jd['sim_bert'] = target_jd['sim_bert'].apply(lambda x: [candidate_jd['job_description'][i] for i in x])
target_jd['sim_edit'] = [i for i in edit_ans]
target_jd['sim_edit_value'] = [i for i in sort_sim_edit]
target_jd['sim_edit'] = target_jd['sim_edit'].apply(lambda x: [candidate_jd['job_description'][i] for i in x])
for col in ['sim_bert', 'sim_edit',	'sim_bert_value','sim_edit_value']:
    target_jd[col] = target_jd[col].apply(lambda x: x[:10])
target_jd = target_jd.drop('bert_task', axis=1)
target_jd.to_excel(f'outputs/{dataset_name}_top10.xlsx', index=None)
target_jd

,job_title,job_description,task,sim_bert,sim_bert_value,sim_bert_raw_value,sim_edit,sim_edit_value
0,"VP, Sr Retail Bus Banker - Illinois Market","Spend majority of time in the field, on sales ...",Contact current or potential customers to prom...,[Operating fiber optic lasers to cut thin-gaug...,"[0.99968386, 0.999651, 0.9996458, 0.9996328, 0...","[0.98956716, 0.9890395, 0.98880047, 0.9886083,...",[Identify problems and improvement opportuniti...,"[167.0, 168.0, 171.0, 176.0, 177.0, 179.0, 179..."
1,Associate Account Executive,Assist in executing marketing client campaigns...,Contact current or potential customers to prom...,[Operating fiber optic lasers to cut thin-gaug...,"[0.999786, 0.99971217, 0.99970573, 0.9997021, ...","[0.99343634, 0.9929291, 0.9928884, 0.99280876,...",[Identify problems and improvement opportuniti...,"[124.0, 126.0, 128.0, 131.0, 131.0, 132.0, 132..."
2,Aftermarket Sales Representative,Develop a territory management plan to maximiz...,Contact current or potential customers to prom...,[Operating fiber optic lasers to cut thin-gaug...,"[0.999814, 0.99968225, 0.9996361, 0.9996212, 0...","[0.99442405, 0.99391925, 0.9937634, 0.9936143,...",[Identify problems and improvement opportuniti...,"[124.0, 129.0, 130.0, 130.0, 131.0, 132.0, 133..."
3,Commercial Lines Producer,"Prospect new business primarily by phone, mail...",Contact current or potential customers to prom...,[Operating fiber optic lasers to cut thin-gaug...,"[0.9997624, 0.9997588, 0.99975747, 0.99975234,...","[0.9922129, 0.9911552, 0.9908361, 0.9905604, 0...",[Identify problems and improvement opportuniti...,"[128.0, 131.0, 132.0, 133.0, 133.0, 133.0, 135..."
4,Regional Sales Development Manager,Effectively communicate voice of customer (VOC...,Contact current or potential customers to prom...,[Operating fiber optic lasers to cut thin-gaug...,"[0.99981266, 0.9998082, 0.9997843, 0.99977845,...","[0.99675184, 0.99654764, 0.9963954, 0.99632597...",[Identify problems and improvement opportuniti...,"[151.0, 152.0, 155.0, 155.0, 155.0, 155.0, 156..."
...,...,...,...,...,...,...,...,...
128,IT Project Manager,- Coordinate with technical leads and develope...,Confer with technical personnel to prepare des...,[Operating fiber optic lasers to cut thin-gaug...,"[0.99975425, 0.99972355, 0.99964076, 0.9996401...","[0.9958055, 0.9956647, 0.99565125, 0.9954921, ...",[Identify problems and improvement opportuniti...,"[102.0, 105.0, 108.0, 109.0, 109.0, 110.0, 111..."
129,Public Relations Manager - Networking,Support content development to promote Cisco s...,Write informational material.,"[Performs salvage, corrective and preventative...","[0.99971044, 0.999694, 0.9996531, 0.9996492, 0...","[0.995232, 0.9939058, 0.99377835, 0.99365836, ...",[Identify problems and improvement opportuniti...,"[104.0, 131.0, 131.0, 132.0, 133.0, 133.0, 133..."
130,Program Security Officer-Journeyman,Develop security plans and procedures for safe...,"Develop safety standards, policies, or procedu...",[Operating fiber optic lasers to cut thin-gaug...,"[0.9991338, 0.9988207, 0.9986209, 0.9985943, 0...","[0.9958207, 0.9952842, 0.9952004, 0.9951075, 0...",[Identify problems and improvement opportuniti...,"[250.0, 254.0, 255.0, 256.0, 257.0, 257.0, 258..."
131,"Director of Finance - The Langham, New York",Direct accounting staff to ensure accurate acc...,"Direct sales, marketing, or customer service a...",[Operating fiber optic lasers to cut thin-gaug...,"[0.99986166, 0.999792, 0.999777, 0.9997614, 0....","[0.9924639, 0.9920392, 0.9913794, 0.9912293, 0...",[Identify problems and improvement opportuniti...,"[128.0, 146.0, 154.0, 157.0, 164.0, 165.0, 165..."
